## Necessary Modules

In [3]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from itertools import product

import os
from glob import glob
import sys
sys.path.append('..')

from sqlalchemy import create_engine, MetaData
from sqlalchemy import Table, Column
from sqlalchemy import Integer, String, Time, DateTime, Numeric, BigInteger
from sqlalchemy import select, insert, update, delete

import yfinance as yf
from litedata import Litedata
Yahoodata = Litedata('yahoo')

from updatesectype_flow import update_sectypedata

## Testing on sectypes

### Reading symbollist CSVs

In [ ]:
#symbolcsvlist = glob(os.path.join(Yahoodata.data_path, 'lite_*.csv'))

#for csvfile in symbolcsvlist:
#    dflist_sectype = pd.read_csv(csvfile, index_col='symbol', header=0)
#    dflist_sectype = dflist_sectype.sort_values(['sector', 'symbol'])
#    dflist_sectype.to_csv(csvfile)

### Enumerating symbols / sectypes

### Remove symbol

In [2]:
#removelist = [symbol for symbol, spec in Yahoodata.symboldict.items() if spec[1] == 'elseetf']
removelist = ['WORK']
for symbol in removelist:
    Yahoodata.remove_symbol(symbol)

### Inserting all symbols from a security type

In [ ]:
#sectypelist_new = ['usstock']
#metadict = {sectype: Yahoodata.get_symboldict_csv(sectype) for sectype in sectypelist_new}
#for sectype in sectypelist_new:
#    metadict[sectype] = Yahoodata.get_symboldict_csv(sectype)
#    Yahoodata.insert_from_symboldict(metadict[sectype])

### Scraping data

In [ ]:
symbollist_test = ['AAPL', 'PYPL', 'TSLA', 'QQQ', 'ARKW', 'EWH', 'DX-Y.NYB', '0388.HK']
numsymbols_test = len(symbollist_test)
startstr_test = '1991-01-01'
endstr_test = '2020-12-31'

datadict_test = Yahoodata.yfinancescrape(symbollist_test, startstr_test, endstr_test)

### Load database symbol data

In [ ]:
df_nvda = Yahoodata.load_dbdata('NVDA')
df_nvda.tail()

### Update Flow (Count time)

In [ ]:
dict_testspec = {
    'Indices': (['^DJI', '^GSPC', '^VIX', '^HSI'], (16,0)),
    'hkstock': (['0388.HK', '0700.HK', '2318.HK', '3690.HK'], (16,0)),
    'sgstock': (['M44U.SI', 'ME8U.SI', 'A17U.SI', 'C2PU.SI'], (17,0)),
    'etf': (['SPY', 'QQQ', 'TLT', 'ARKK'], (16,0)),
    'levetf': (['TQQQ', 'SPXL', 'TMF', 'TECL'], (16,0)) ,
    'usstock': (['IOVA', 'MELI', 'SE', 'SQ'], (16,0)),
    'largecap': (['AAPL', 'AMZN', 'MSFT', 'NVDA'], (16,0))
}

sectypelist_asia = ['hkstock', 'sgstock']
sectypelist_us = ['usstock', 'largecap', 'etf', 'levetf']

In [ ]:
for sectype in sectypelist_us:
    testspec = dict_testspec[sectype]
    update_sectypedata(sectype, testspec[0], testspec[1], 18, 5)

In [4]:
#with Yahoodata.engine.connect() as con:
#    stmt_update = f'UPDATE `Indices_^HSI` SET `date` = "{datetime(2019, 12, 18, 16, 0, 0)}" \
#                    WHERE `date` between "{datetime(2019, 12, 18, 0, 0, 0)}" and "{datetime(2019, 12, 18, 1, 0, 0)}"'
#    con.execute(stmt_update)     

In [5]:
#with Yahoodata.engine.connect() as con:
#    meta = MetaData()
#    insertlist = [{'vendor': 'yahoo', 'date': datetime(2020, 3, 23, 16, 0, 0),
#                    'op': 21659.55, 'hi': 22062.19, 'lo': 21659.55, 'cl': 21696.13, 'vol': 3478982700,
#                    'adj_op': 21659.55, 'adj_hi': 22062.19, 'adj_lo': 21659.55, 'adj_cl': 21696.13, 'adj_vol': 3478982700,
#                    'div': 0, 'split': 0, 'update': datetime.now()}] 
#    table_ohlc = Table('Indices_^HSI', meta, autoload=True, autoload_with=Yahoodata.engine)
#    insertquery = insert(table_ohlc)
#    con.execute(insertquery, insertlist)

In [ ]:
#insertlist = [{'vendor': 'yahoo', 'date': datetime(2019, 12, 18),
#                'op': 27946.74, 'hi': 27970.54, 'lo': 27777.47, 'cl': 27884.21, 'vol': 1842227300,
#                'adj_op': 27946.74, 'adj_hi': 27970.54, 'adj_lo': 27777.47, 'adj_cl': 27884.21, 'adj_vol': 1842227300,
#                'div': 0, 'split': 0, 'update': datetime.now()}] 